## 0. Libarary 불러오기 및 경로설정

In [1]:
!nvidia-smi

Thu Apr  8 08:19:25 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P40           On   | 00000000:00:05.0 Off |                  Off |
| N/A   51C    P0   156W / 250W |   6693MiB / 24451MiB |     79%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import glob
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
from efficientnet_pytorch import EfficientNet
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from adamp import AdamP
from madgrad import MADGRAD
import albumentations as A
from tqdm import tqdm
import random
from sklearn.metrics import f1_score
from utils import GetPathNLabel


In [3]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore
    random.seed(seed)

In [4]:
seed_everything(42)

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
test_dir = '/opt/ml/input/data/eval'
train_dir = '/opt/ml/input/data/train'
train_image_dir = '/opt/ml/input/data/train/images'

In [7]:
df = pd.read_csv(os.path.join(train_dir, 'train.csv'))

In [8]:
temp_label = []
for i in range(len(df)):
    temp_gender = df.iloc[i].gender
    temp_age = df.iloc[i].age
    gender_label = 0 if temp_gender == 'male' else 3
    age_label = 0 if temp_age < 30 else 1 if temp_age >= 30 and temp_age < 60 else 2
    temp_label.append(gender_label + age_label)

df_label = pd.Series(temp_label)
df_label_add = pd.concat((df, df_label), axis=1)
df_label_add.rename(columns={0:'label'}, inplace=True)
temp_label = df_label_add['label'].to_list()
temp_folder = df_label_add['path'].to_list()


## 1. Model 정의

In [12]:
from model import EffNet

## 2. Test Dataset 정의

In [13]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform, aug=None):
        self.img_paths = img_paths
        self.transform = transform
        self.aug = aug
    def __getitem__(self, index):
        # image = Image.open(self.img_paths[index])
        image = cv2.imread(self.img_paths[index], cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # image = (image/255.)
        img_ycrcb = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        
        ycrcb_planes = cv2.split(img_ycrcb)
        # 밝기 성분에 대해서만 histogram equalization
        ycrcb_planes[0] = cv2.equalizeHist(ycrcb_planes[0])
        dst_ycrcb = cv2.merge(ycrcb_planes)
        dst = cv2.cvtColor(dst_ycrcb, cv2.COLOR_YCrCb2RGB)
        dst = (dst/255.).astype('float32')

        if self.aug:
            dst = self.aug(image=dst)['image']
        if self.transform:
            dst = self.transform(dst)
        
        return dst

    def __len__(self):
        return len(self.img_paths)

## Train

In [14]:
from utils import MaskDataset

In [16]:
class F1Loss(nn.Module):
    def __init__(self, classes=18, epsilon=1e-7):
        super().__init__()
        self.classes = classes
        self.epsilon = epsilon
    def forward(self, y_pred, y_true):
        assert y_pred.ndim == 2
        assert y_true.ndim == 1
        y_true = F.one_hot(y_true, self.classes).to(torch.float32)
        y_pred = F.softmax(y_pred, dim=1)

        tp = (y_true * y_pred).sum(dim=0).to(torch.float32)
        tn = ((1 - y_true) * (1 - y_pred)).sum(dim=0).to(torch.float32)
        fp = ((1 - y_true) * y_pred).sum(dim=0).to(torch.float32)
        fn = (y_true * (1 - y_pred)).sum(dim=0).to(torch.float32)

        precision = tp / (tp + fp + self.epsilon)
        recall = tp / (tp + fn + self.epsilon)

        f1 = 2 * (precision * recall) / (precision + recall + self.epsilon)
        f1 = f1.clamp(min=self.epsilon, max=1 - self.epsilon)
        return 1 - f1.mean()

In [17]:
class FocalLoss(nn.Module):
    def __init__(self, weight=None,
                 gamma=2., reduction='mean'):
        nn.Module.__init__(self)
        self.weight = weight
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, input_tensor, target_tensor):
        log_prob = F.log_softmax(input_tensor, dim=-1)
        prob = torch.exp(log_prob)
        return F.nll_loss(
            ((1 - prob) ** self.gamma) * log_prob,
            target_tensor,
            weight=self.weight,
            reduction=self.reduction
        )


## MIXUP

In [20]:
model_save_path = os.path.join(os.getcwd(), 'model', 'day11/')
models = []
skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
best_models = []
for fold_index, (train_idx, val_idx) in enumerate(skfold.split(temp_folder, temp_label)):
    # if fold_index == 0 or fold_index == 1 or fold_index == 2 or fold_index == 3:
    #     continue
    print(f'[fold: {fold_index}]')
    torch.cuda.empty_cache()

    augmentations = A.Compose([
        A.RandomBrightnessContrast(p=0.3),
        # A.Rotate(limit=20, p=0.3),
        # A.HorizontalFlip(p=0.3),
        # A.VerticalFlip(p=0.3),
        # A.IAAPiecewiseAffine(p=0.3),
        # A.RandomRotate90(p=0.2),
        # A.RandomGamma(p=0.2),
        # A.GaussNoise(),
        # A.GaussianBlur(blur_limit=(3,5)),
        A.HueSaturationValue(p=0.3),
        # A.RGBShift()
    ])
    train_dirs = [temp_folder[i] for i in train_idx]
    val_dirs = [temp_folder[i] for i in val_idx]
    


    ############################################################
    # train_images = [images[i] for i in train_idx]
    # train_labels = [labels[i] for i in train_idx]

    # val_images = [images[i] for i in val_idx]
    # val_labels = [labels[i] for i in val_idx]
    train_path = GetPathNLabel(train_dirs)
    val_path = GetPathNLabel(val_dirs)
    train_images, train_labels = train_path.call()
    val_images, val_labels = val_path.call()
    

    train_dataset = MaskDataset(train_images, train_labels, augmentations=augmentations)
    val_dataset = MaskDataset(val_images, val_labels)
    train_data_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=3)
    val_data_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=3)
    
    model = EffNet()
    # model._dropout = nn.Dropout(p=0.7, inplace=False)
    model.to(device)
    # optimizer = AdamP(model.parameters(), lr=0.0001)
    optimizer = MADGRAD(model.parameters(), lr=0.0001)
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
    # lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=4, eta_min=0.000005)
    criterion1 = FocalLoss(gamma=5)
    criterion2 = F1Loss()
    #################
    valid_f1_max = 0
    valid_loss_min = float('inf')
    best_model = None
    for epoch in range(15):
            # train_acc_list = []
            train_f1_list = []
            train_loss_list = []
            with tqdm(train_data_loader,
                    total=train_data_loader.__len__(), 
                    unit="batch") as train_bar:
                for sample in train_bar:
                    train_bar.set_description(f"Train Epoch {epoch}")
                
                    optimizer.zero_grad()
                    x_train, y_train = sample['image'], sample['label']
                    x_train = x_train.to(device)
                    y_train = y_train.to(device)
                    model.train()
                    with torch.set_grad_enabled(True):
                        preds  = model(x_train)
                        loss = (0.5*criterion1(preds, y_train)) + (0.5*criterion2(preds, y_train))
                        preds = preds.argmax(dim=-1)

                        loss.backward()
                        optimizer.step()

                        preds  = preds.cpu().detach().numpy()
                        y_train = y_train.cpu().detach().numpy()
                        # batch_acc = (y_train == preds).mean()    
                        # train_acc_list.append(batch_acc)
                        # train_acc = np.mean(train_acc_list)
                        batch_f1 = f1_score(y_train, preds, average='macro')
                        train_f1_list.append(batch_f1)
                        train_loss_list.append(loss.item())
                        

                        train_f1 = np.mean(train_f1_list)
                        train_loss = np.mean(train_loss_list)
                        train_bar.set_postfix(train_loss= train_loss,
                                        train_f1 = train_f1)
                    
                    

            # valid_acc_list = []
            valid_f1_list = []
            valid_loss_list = []
            with tqdm(val_data_loader,
                    total=val_data_loader.__len__(),
                    unit="batch") as valid_bar:
                for sample in valid_bar:
                    valid_bar.set_description(f"Valid Epoch {epoch}")
                    optimizer.zero_grad()
                    x_val, y_val = sample['image'], sample['label']
                    x_val = x_val.to(device)
                    y_val = y_val.to(device)

                    model.eval()
                    with torch.no_grad():
                        preds = model(x_val)
                        valid_loss = (0.5*criterion1(preds, y_val)) + (0.5*criterion2(preds, y_val))
                        preds = preds.argmax(dim=-1)

                        preds  = preds.cpu().detach().numpy()
                        y_val = y_val.cpu().detach().numpy()
                        # batch_acc = (y_val == preds).mean()
                        # valid_acc_list.append(batch_acc)
                        batch_f1 = f1_score(y_val, preds, average='macro')
                        valid_f1_list.append(batch_f1)
                        valid_loss_list.append(valid_loss.item())
                        # valid_acc = np.mean(valid_acc_list)
                        valid_f1 = np.mean(valid_f1_list)
                        valid_loss = np.mean(valid_loss_list)
                        valid_bar.set_postfix(valid_loss = valid_loss,
                                        valid_f1 = valid_f1)

                    
                    
                
            lr_scheduler.step()

            if valid_f1_max < valid_f1:
                print()
                print(f"best model changed!!, previous: {valid_f1_max} VS current: {valid_f1}")
                valid_f1_max = valid_f1
                best_model = model
                best_epoch = epoch
        
    model_name = "Effnetb3"
    path = model_save_path
    torch.save(best_model, f'{model_save_path}{model_name}_{fold_index}_{valid_f1_max:2.4f}_epoch_{best_epoch}.pth')
    models.append(best_model)

SyntaxError: invalid syntax (<ipython-input-20-d131b97f7897>, line 21)

In [18]:
# models = []
# models.append(torch.load('./model/day9/Effnetb3_0_0.7576_epoch_9.pth'))
# models.append(torch.load('./model/day9/Effnetb3_1_0.7506_epoch_8.pth'))
# models.append(torch.load('./model/day9/Effnetb3_2_0.7502_epoch_11.pth'))
# models.append(torch.load('./model/day9/Effnetb3_3_0.7506_epoch_8.pth'))


## 3. Inference

In [19]:
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]

In [20]:
image_paths[:10]

['/opt/ml/input/data/eval/images/cbc5c6e168e63498590db46022617123f1fe1268.jpg',
 '/opt/ml/input/data/eval/images/0e72482bf56b3581c081f7da2a6180b8792c7089.jpg',
 '/opt/ml/input/data/eval/images/b549040c49190cedc41327748aeb197c1670f14d.jpg',
 '/opt/ml/input/data/eval/images/4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg',
 '/opt/ml/input/data/eval/images/248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg',
 '/opt/ml/input/data/eval/images/7953c2e6e983bad91b89a0e4ad7eb69e6e43e56b.jpg',
 '/opt/ml/input/data/eval/images/1903db7dcc1262d0441677afa422e6a8371e5b37.jpg',
 '/opt/ml/input/data/eval/images/441419a874f4d031cd576850b68539ca7d35bedf.jpg',
 '/opt/ml/input/data/eval/images/388856cd1ef99b1918273a827a75f2aff2478321.jpg',
 '/opt/ml/input/data/eval/images/795ba8ccc769a3f9da6a897f75df6706b729345b.jpg']

In [21]:
len(image_paths)

12600

In [22]:
# meta 데이터와 이미지 경로를 불러옵니다.
preds = []
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]

for model in models:
    transform = transforms.Compose([
        # Resize((512, 384), Image.BILINEAR),
        transforms.ToTensor(),
        transforms.ToPILImage(),
        transforms.CenterCrop((300, 256)),

        transforms.ToTensor(),
        Normalize(mean=(0.560, 0.524, 0.501), std=(0.233, 0.243, 0.245)),
        # transforms.ToTensor(),
        # transforms.ToPILImage(),
        # transforms.ToTensor()
    ])
    dataset = TestDataset(image_paths, transform)

    loader = DataLoader(
        dataset,
        batch_size=32,
        shuffle=False,
        num_workers=3,
        drop_last=False
    )

    # 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
    device = torch.device('cuda')
    # model = torch.load('./model/Effnetb2_0.0788_epoch_2.pth')
    model.eval()

    # 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
    all_predictions = []
    for images in loader:
        with torch.no_grad():
            images = images.to(device)
            pred = model(images)
            # preds.append(pred)
            # pred = pred.argmax(dim=-1)
            all_predictions.extend(pred.cpu().numpy())
            # print(len(all_predictions))

    preds.append(all_predictions)
print("done")

done


In [23]:
pred_np = np.array(preds)

In [24]:
pred_np.shape

(7, 12600, 18)

In [25]:
pred_np_sum = pred_np.sum(axis=0)
pred_np_mean = np.mean(pred_np_sum, axis=-1)
final = np.argmax(pred_np_sum, axis=-1)

In [26]:
submission['ans'] = final
# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission_final4.csv'), index=False)
print('test inference is done!')

test inference is done!
